# Spotify Song Data Analysis
## Introduction
This is my capstone 2, spotify song database
## Importing Libraries

In [4]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;

## Reading the dataset

In [10]:
df = pd.read_csv('data\data.csv');
dfArtist = pd.read_csv('data\data_by_artist.csv');
dfGenre = pd.read_csv('data\data_by_genres.csv');
dfYear = pd.read_csv('data\data_by_year.csv');
dfWGenres = pd.read_csv('data\data_w_genres.csv');

In [16]:
data_list = [df, dfArtist, dfGenre, dfYear, dfWGenres]

for i in data_list:
    print('-' * 100);
    print(i.describe());
    print('-' * 100);

----------------------------------------------------------------------------------------------------
             valence           year   acousticness   danceability  \
count  170653.000000  170653.000000  170653.000000  170653.000000   
mean        0.528587    1976.787241       0.502115       0.537396   
std         0.263171      25.917853       0.376032       0.176138   
min         0.000000    1921.000000       0.000000       0.000000   
25%         0.317000    1956.000000       0.102000       0.415000   
50%         0.540000    1977.000000       0.516000       0.548000   
75%         0.747000    1999.000000       0.893000       0.668000   
max         1.000000    2020.000000       0.996000       0.988000   

        duration_ms         energy       explicit  instrumentalness  \
count  1.706530e+05  170653.000000  170653.000000     170653.000000   
mean   2.309483e+05       0.482389       0.084575          0.167010   
std    1.261184e+05       0.267646       0.278249          0.313

In [17]:
for i in data_list:
    print('-' * 100);
    print(i.head());
    print('-' * 100);

----------------------------------------------------------------------------------------------------
   valence  year  acousticness  \
0   0.0594  1921         0.982   
1   0.9630  1921         0.732   
2   0.0394  1921         0.961   
3   0.1650  1921         0.967   
4   0.2530  1921         0.957   

                                             artists  danceability  \
0  ['Sergei Rachmaninoff', 'James Levine', 'Berli...         0.279   
1                                     ['Dennis Day']         0.819   
2  ['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...         0.328   
3                                   ['Frank Parker']         0.275   
4                                     ['Phil Regan']         0.418   

   duration_ms  energy  explicit                      id  instrumentalness  \
0       831667   0.211         0  4BJqT0PrAfrxzMOxytFOIz          0.878000   
1       180533   0.341         0  7xPhfUan2yNtyFG0cUWkt8          0.000000   
2       500062   0.166         0  1o6I8